In [4]:
######## Green areas without playgrounds #####

import json
import folium
import shapely.geometry as sg

# Load the JSON data
with open('arealanvendelse.json') as f:
    data = json.load(f)

# Create the Folium map
latitude = 55.676  # Example latitude
longitude = 12.568  # Example longitude
zoom_start = 13  # Initial zoom level
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

# Extract areas and add them as polygons to the map
for feature in data['features']:
    geometry = sg.asShape(feature['geometry'])
    if geometry.geom_type == 'Polygon':
        coordinates = [[coord[1], coord[0]] for coord in geometry.exterior.coords[:]]  # Flipping coordinates
        folium.Polygon(locations=coordinates, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(m)
    elif geometry.geom_type == 'MultiPolygon':
        for polygon in geometry:
            coordinates = [[coord[1], coord[0]] for coord in polygon.exterior.coords[:]]  # Flipping coordinates
            folium.Polygon(locations=coordinates, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(m)

# Display the map
m


AttributeError: module 'shapely.geometry' has no attribute 'asShape'

In [5]:
#### FØRSTE METODE TIL PERCENTAGE GREEN AREAS

import json
from shapely.geometry import shape, Polygon

# Load neighborhood data
with open('bydel.json') as f:
    neighborhood_data = json.load(f)

# Load green area data
with open('arealanvendelse.json') as f:
    green_area_data = json.load(f)

# Convert neighborhood data to Shapely polygons
neighborhood_polygons = [shape(feature['geometry']).buffer(0) for feature in neighborhood_data['features']]

# Convert green area data to Shapely polygons
green_area_polygons = [shape(feature['geometry']).buffer(0) for feature in green_area_data['features']]

# Initialize list to store green area within each neighborhood
green_area_within_neighborhoods = []

# Calculate area of green spaces within each neighborhood
for neighborhood_polygon in neighborhood_polygons:
    intersection_area = 0
    for green_polygon in green_area_polygons:
        intersection = neighborhood_polygon.intersection(green_polygon)
        if intersection.is_empty:
            continue
        if intersection.geom_type == 'Polygon':
            intersection_area += intersection.area
        elif intersection.geom_type == 'MultiPolygon':
            intersection_area += sum(part.area for part in intersection.geoms)
            # intersection_area += sum(part.area for part in intersection)
    green_area_within_neighborhoods.append(intersection_area)

# Calculate total area of each neighborhood
neighborhood_areas = [polygon.area for polygon in neighborhood_polygons]

# Calculate percentage of green area coverage for each neighborhood
percentage_green_area_coverage = [(green_area / neighborhood_area) * 100 
                                  for green_area, neighborhood_area in zip(green_area_within_neighborhoods, neighborhood_areas)]

# Display results
for i, feature in enumerate(neighborhood_data['features']):
    neighborhood_name = feature['properties']['navn']
    green_coverage_percentage = percentage_green_area_coverage[i]
    print(f"{neighborhood_name}: {green_coverage_percentage:.2f}%")


Østerbro: 13.75%
Indre By: 13.19%
Vanløse: 15.03%
Brønshøj-Husum: 34.40%
Amager Øst: 19.71%
Valby: 21.24%
Amager Vest: 51.58%
Bispebjerg: 31.54%
Nørrebro: 11.41%
Vesterbro-Kongens Enghave: 22.87%


# NONE areas:

In [11]:
import json
import folium
import shapely.geometry as sg

# Load JSON data from file
with open('arealanvendelse.json') as f:
    data_arealanvendelse = json.load(f)

# Create the Folium map
latitude = 55.676  # Example latitude
longitude = 12.568  # Example longitude
zoom_start = 13  # Initial zoom level
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)


for feature in data_arealanvendelse['features']:
    objekt_type = feature['properties'].get('objekt_type')
    if objekt_type is None:
        geometry = shape(feature['geometry'])
        if geometry.geom_type == 'Polygon':
            coordinates = [[coord[1], coord[0]] for coord in geometry.exterior.coords[:]]  # Flipping the coordinates
            folium.Polygon(locations=coordinates, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(m)
        elif geometry.geom_type == 'MultiPolygon':
            for polygon in geometry.geoms:
                coordinates = [[coord[1], coord[0]] for coord in polygon.exterior.coords[:]]  # Flipping coordinates
                folium.Polygon(locations=coordinates, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(m)

# Display the map
m
